In [101]:
!pip install --quiet cloudscraper requests PyMuPDF Pillow selenium youtube-transcript-api google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client replicate cohere python-dotenv pymilvus



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [203]:
import os
from dotenv import load_dotenv
from pymilvus import MilvusClient, DataType
load_dotenv()

zilliz_endpoint = os.getenv("ZILLIZ_ENDPOINT")
zilliz_token = os.getenv("ZILLIZ_TOKEN")
os.environ["REPLICATE_API_TOKEN"] = os.getenv("REPLICATE_API_TOKEN")
COHERE_API_KEY=os.getenv("COHERE_API_KEY")

In [186]:
import uuid
import time
import json
from datetime import datetime
from langchain_text_splitters import RecursiveCharacterTextSplitter
import cohere
import replicate
import requests
from PIL import Image, UnidentifiedImageError
import io
import tempfile
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

In [150]:
import re
import cloudscraper
import time
import pickle
import requests
import fitz  # PyMuPDF
from PIL import Image
import io
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from youtube_transcript_api import YouTubeTranscriptApi
from google.oauth2.credentials import Credentials
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload

# Function to extract YouTube video ID from URL
def get_youtube_video_id(url):
    pattern = r"(?:v=|\/)([0-9A-Za-z_-]{11}).*"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        raise ValueError("Invalid YouTube URL")

# Function to extract transcript text from YouTube video ID
def get_transcript_text(video_id):
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    full_text = ' '.join([entry['text'] for entry in transcript])
    return full_text

# Function to get transcript from YouTube URL
def yt_transcript(yt_url):
    try:
        video_id = get_youtube_video_id(yt_url)
        transcript_text = get_transcript_text(video_id)
        return transcript_text
    except Exception as e:
        return str(e)

# Function to authenticate and upload to Google Drive
def authenticate_drive():
    SCOPES = ['https://www.googleapis.com/auth/drive']
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    drive_service = build('drive', 'v3', credentials=creds)
    return drive_service

# Function to combine images vertically
def combine_images(images):
    widths, heights = zip(*(img.size for img in images))
    total_height = sum(heights)
    max_width = max(widths)
    combined_image = Image.new('RGB', (max_width, total_height))
    y_offset = 0
    for img in images:
        combined_image.paste(img, (0, y_offset))
        y_offset += img.height
    return combined_image

# Function to download PDF from a URL
def download_pdf(pdf_url):
    response = requests.get(pdf_url)
    pdf_data = io.BytesIO(response.content)
    return pdf_data

# Function to process PDF and upload combined images to Google Drive
def process_and_upload(pdf_url, title):
    pdf_data = download_pdf(pdf_url)
    doc = fitz.open(stream=pdf_data, filetype="pdf")
    
    drive_service = authenticate_drive()
    
    # Folder ID of the parent folder where new folder will be created
    parent_folder_id = '1VVE6uwli8pE-T8QrsNRirSb604rWHPs4'
    
    # Create a new folder inside the parent folder
    folder_metadata = {
        'name': f'{title} Presentation',
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [parent_folder_id]
    }
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    folder_id = folder.get('id')
    
    images_to_combine = []
    image_index = 0
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        images_to_combine.append(img)

        if len(images_to_combine) == 3:
            combined_image = combine_images(images_to_combine)
            image_filename = f"slide_{image_index + 1}.png"

            # Save combined image to a BytesIO object
            image_stream = io.BytesIO()
            combined_image.save(image_stream, format='PNG')
            image_stream.seek(0)  # Reset stream position

            # Upload to Google Drive directly from memory
            file_metadata = {
                'name': image_filename,
                'parents': [folder_id]
            }
            media = MediaIoBaseUpload(image_stream, mimetype='image/png')
            drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

            images_to_combine = []  # Reset the list for the next batch
            image_index += 1

    # Process any remaining images
    if images_to_combine:
        combined_image = combine_images(images_to_combine)
        image_filename = f"slide_{image_index + 1}.png"

        image_stream = io.BytesIO()
        combined_image.save(image_stream, format='PNG')
        image_stream.seek(0)

        file_metadata = {
            'name': image_filename,
            'parents': [folder_id]
        }
        media = MediaIoBaseUpload(image_stream, mimetype='image/png')
        drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    folder_link = f"https://drive.google.com/drive/folders/{folder_id}"
    return folder_link

# Function to scrape all URLs from the main library page
def scrape_main_library_url(main_url):
    try:
        scraper = cloudscraper.create_scraper()
        response = scraper.get(main_url)
        cookies = response.cookies

        # Set up WebDriver (using Chrome in headless mode)
        chrome_options = Options()
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")
        chrome_options.add_argument("--log-level=3")
        chrome_options.add_argument("--silent")

        service = Service('./chromedriver.exe')
        driver = webdriver.Chrome(service=service, options=chrome_options)

        driver.get(main_url)
        for cookie in cookies:
            driver.add_cookie({'name': cookie.name, 'value': cookie.value, 'domain': cookie.domain})

        driver.get(main_url)
        time.sleep(5)  # Adjust time as needed

        wait = WebDriverWait(driver, 40)

        xpaths = "//a[@class='link-black']"
        urls = []

        elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpaths)))
        urls = [element.get_attribute('href') for element in elements]

        print(f"Found {len(urls)} presentation URLs.")
        return urls
    except Exception as e:
        print(f"Error while scraping main library URL: {str(e)}")
        return []
    finally:
        driver.quit()

# Function to scrape content from a single page
def scrape_single_page(url):
    # Set up WebDriver (using Chrome in headless mode)
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_argument("--silent")

    service = Service('./chromedriver.exe')
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        # Navigate to the target webpage
        driver.get(url)
        wait = WebDriverWait(driver, 10)

        # Check if the page is "top-rated" by looking for the star image
        try:
            star_xpath = "/html/body/div[1]/main/div/div[1]/div[2]/div/div/div/div[3]/div/div[2]/img"
            driver.find_element(By.XPATH, star_xpath)
            is_top_rated = True
            print("Top-rated page detected.")
        except:
            is_top_rated = False

        # Use alternate XPaths if top-rated, otherwise use standard XPaths
        if is_top_rated:
            title_xpath = "/html/body/div[1]/main/div/div[1]/div[2]/div/div/div/div[3]/div/div[2]/h2"
            date_xpath = "/html/body/div[1]/main/div/div[2]/div/div/div[1]/div/div[1]/div/div/p/time"
            text_xpath = "/html/body/div[1]/main/div/div[2]/div/div/div[1]/div/div[2]/div/div/div[3]"
            video_xpath = "/html/body/div[1]/main/div/div[2]/div/div/div[1]/div/div[3]/div/div/div/div/div/div/iframe"
            presentation_xpath = "/html/body/div[1]/main/div/div[2]/div/div/div[1]/div/div[4]/div/div/a"
        else:
            title_xpath = "/html/body/div[1]/main/div/div/div[2]/div/div/div/div[3]/div/div[2]/h2"
            date_xpath = "/html/body/div[1]/main/div/div/div[3]/div/div/div[1]/div/div[1]/div/div/p/time"
            text_xpath = "/html/body/div[1]/main/div/div/div[3]/div/div/div[1]/div/div[2]"
            video_xpath = "/html/body/div[1]/main/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/div/div/div/div/iframe"
            presentation_xpath = "/html/body/div[1]/main/div/div/div[3]/div/div/div[1]/div/div[4]/div/div/a"

        # Extract the title
        title_element = wait.until(EC.presence_of_element_located((By.XPATH, title_xpath)))
        title = title_element.text

        # Extract the date
        date_element = wait.until(EC.presence_of_element_located((By.XPATH, date_xpath)))
        date = date_element.text

        # Extract the text content
        text_element = wait.until(EC.presence_of_element_located((By.XPATH, text_xpath)))
        text = text_element.text

        # Extract the YouTube video URL
        try:
            video_element = driver.find_element(By.XPATH, video_xpath)
            video_url = video_element.get_attribute('src')
        except:
            video_url = "NA"

        # Extract the presentation download link
        try:
            presentation_element = driver.find_element(By.XPATH, presentation_xpath)
            presentation_url = presentation_element.get_attribute('href')
        except:
            presentation_url = "NA"

        # Return the result as a dictionary
        return {
            "url": url,
            "title": title,
            "date": date,
            "text": text,
            "video": video_url,
            "presentation": presentation_url
        }

    except Exception as e:
        print(f"Error while scraping {url}: {str(e)}")
        return None

    finally:
        driver.quit()
        print("driver closed")

# Function to get YouTube transcripts and add to dicts
def add_transcripts_to_dicts(data_list):
    for data in data_list:
        if data.get("video") and data["video"] != "NA":
            print(f"Getting transcript for video: {data['video']}")
            transcript = yt_transcript(data["video"])
            data["transcript"] = transcript
    print("Got the YouTube video transcripts!")

# Function to upload presentation slides to Google Drive and add link to dicts
def upload_presentations_to_drive(data_list):
    for data in data_list:
        if data.get("presentation") and data["presentation"] != "NA":
            print(f"Uploading presentation slides for: {data['title']}")
            drive_link = process_and_upload(data["presentation"], data["title"])
            data["drive"] = drive_link
    print("Uploaded presentation slides to drive!")
   


In [7]:
data_list = []

In [9]:
main_url = "https://www.rsaconference.com/library#sort=%40industrytopiccontentdate%20descending&numberOfResults=25&f:contenttype=[Presentation]"
urls = scrape_main_library_url(main_url)
if not urls:
    print("No URLs found.")

Found 25 presentation URLs.


In [9]:
print(urls)

['https://www.rsaconference.com/library/presentation/usa/2024/closing%20celebration%20with%20alicia%20keys', 'https://www.rsaconference.com/library/presentation/usa/2024/the%20one%20about%20controls', 'https://www.rsaconference.com/library/presentation/usa/2024/kubernetes%20security%20attacking%20and%20defending%20modern%20infrastructure', 'https://www.rsaconference.com/library/presentation/usa/2024/fud%20vs%20reality%20bracing%20for%20an%20ai-powered%20threat%20landscape', 'https://www.rsaconference.com/library/presentation/usa/2024/project%20upskill%20digital%20security%20for%20high-risk%20communities', 'https://www.rsaconference.com/library/presentation/usa/2024/data%20heist%20how%20stolen%20information%20becomes%20a%20hot%20commodity', 'https://www.rsaconference.com/library/presentation/usa/2024/the%20art%20of%20probable%20test%20with%20ai%20atomic%20red%20team%20and%20threat%20metrics', 'https://www.rsaconference.com/library/presentation/usa/2024/the%20canary%20in%20the%20coalmine

In [78]:
page_data = scrape_single_page(urls[24])
if page_data:
    data_list.append(page_data)
print("Successfully scraped the page.")


driver closed
Successfully scraped the page.


In [79]:
print(len(data_list))

25


In [80]:
print(data_list)

[{'url': 'https://www.rsaconference.com/library/presentation/usa/2024/closing%20celebration%20with%20alicia%20keys', 'title': 'Closing Celebration with Alicia Keys', 'date': 'May 09, 2024', 'text': 'Join us as we celebrate the closing of RSAC 2024. Priority will be given to Full Conference, Expo Plus, Speaker, and Media Pass holders, with limited seating available for College Day, Expo, and Booth Staff Pass holders.', 'video': 'https://www.youtube.com/embed/-P1GCAP04rY', 'presentation': 'NA'}, {'url': 'https://www.rsaconference.com/library/presentation/usa/2024/the%20one%20about%20controls', 'title': 'The One About Controls', 'date': 'May 09, 2024', 'text': 'A practitioner’s take on Maslow’s hierarchy of needs and how that plays into our innate need to control. In this session, presenters will speak about how to use a library of controls while they define, implement and measure and periodically validate security effectiveness for meeting compliance needs and transforming the security m

In [81]:
add_transcripts_to_dicts(data_list)


Getting transcript for video: https://www.youtube.com/embed/-P1GCAP04rY
Getting transcript for video: https://www.youtube.com/embed/yxQ7Kdfyvg8
Getting transcript for video: https://www.youtube.com/embed/iCzUv6-9zrI
Getting transcript for video: https://www.youtube.com/embed/dlAJChYL8e0
Getting transcript for video: https://www.youtube.com/embed/LP4w8Wo4OlU
Getting transcript for video: https://www.youtube.com/embed/SmuLWMw4faY
Getting transcript for video: https://www.youtube.com/embed/GSYBgLABG6M
Getting transcript for video: https://www.youtube.com/embed/5XnRxpgFcKg
Getting transcript for video: https://www.youtube.com/embed/lAiOT8ofWqA
Getting transcript for video: https://www.youtube.com/embed/h0i3Iwd0aIs
Getting transcript for video: https://www.youtube.com/embed/DnwXD4xUi-k
Getting transcript for video: https://www.youtube.com/embed/E7xVBw1xi1k
Getting transcript for video: https://www.youtube.com/embed/xoyPPGivPQc
Getting transcript for video: https://www.youtube.com/embed/_Efp

In [88]:
print(data_list[1]['presentation'])

https://static.rainfocus.com/rsac/us24/sess/1697206824344001paw5/finalwebsite/2024_USA24_GRC-R06_01_The-One-About-Controls_1713893627531001chZL.pdf


In [89]:
upload_presentations_to_drive(data_list)

Uploading presentation slides for: The One About Controls
Uploading presentation slides for: Kubernetes Security: Attacking and Defending Modern Infrastructure
Uploading presentation slides for: FUD vs. Reality: Bracing for an AI-Powered Threat Landscape
Uploading presentation slides for: Project Upskill: Digital Security for High-Risk Communities
Uploading presentation slides for: Data Heist: How Stolen Information Becomes a Hot Commodity
Uploading presentation slides for: The ART of Probable: Test with AI, Atomic Red Team, and Threat Metrics
Uploading presentation slides for: The Canary in the Coalmine of Cybersecurity: Fraud as a Warning Sign
Uploading presentation slides for: A National Initiative for Cybersecurity Advancement
Uploading presentation slides for: Techniques for Automatic Device Identification and Network Assignment
Uploading presentation slides for: Secure Modern Data Lakes - A Primer
Uploading presentation slides for: The Odd One Out - Unleashing the Power of the Un

In [182]:
print(data_list[0])

{'url': 'https://www.rsaconference.com/library/presentation/usa/2024/closing%20celebration%20with%20alicia%20keys', 'title': 'Closing Celebration with Alicia Keys', 'date': 'May 09, 2024', 'text': 'Join us as we celebrate the closing of RSAC 2024. Priority will be given to Full Conference, Expo Plus, Speaker, and Media Pass holders, with limited seating available for College Day, Expo, and Booth Staff Pass holders.', 'video': 'https://www.youtube.com/embed/-P1GCAP04rY', 'presentation': 'NA', 'transcript': '\nCould not retrieve a transcript for the video https://www.youtube.com/watch?v=-P1GCAP04rY! This is most likely caused by:\n\nSubtitles are disabled for this video\n\nIf you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the

In [183]:
print(data_list)

[{'url': 'https://www.rsaconference.com/library/presentation/usa/2024/closing%20celebration%20with%20alicia%20keys', 'title': 'Closing Celebration with Alicia Keys', 'date': 'May 09, 2024', 'text': 'Join us as we celebrate the closing of RSAC 2024. Priority will be given to Full Conference, Expo Plus, Speaker, and Media Pass holders, with limited seating available for College Day, Expo, and Booth Staff Pass holders.', 'video': 'https://www.youtube.com/embed/-P1GCAP04rY', 'presentation': 'NA', 'transcript': '\nCould not retrieve a transcript for the video https://www.youtube.com/watch?v=-P1GCAP04rY! This is most likely caused by:\n\nSubtitles are disabled for this video\n\nIf you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate th

In [184]:
print(data_list[1])

{'url': 'https://www.rsaconference.com/library/presentation/usa/2024/the%20one%20about%20controls', 'title': 'The One About Controls', 'date': 'May 09, 2024', 'text': 'A practitioner’s take on Maslow’s hierarchy of needs and how that plays into our innate need to control. In this session, presenters will speak about how to use a library of controls while they define, implement and measure and periodically validate security effectiveness for meeting compliance needs and transforming the security maturity.', 'video': 'https://www.youtube.com/embed/yxQ7Kdfyvg8', 'presentation': 'https://static.rainfocus.com/rsac/us24/sess/1697206824344001paw5/finalwebsite/2024_USA24_GRC-R06_01_The-One-About-Controls_1713893627531001chZL.pdf', 'transcript': '- Good afternoon everybody. I\'d like to welcome you to\nour next session titled, "The One About Control." Please help me welcome our speaker, Sri, Deputy CISO at Genpact, to the stage. (audience claps) - Thank you everyone. Thanks for making it here. 

In [152]:
import uuid
import time
from datetime import datetime
from langchain_text_splitters import RecursiveCharacterTextSplitter
import cohere

In [166]:
# Set up a Cohere client
cohere_client = cohere.Client(COHERE_API_KEY)

def get_embeddings(text):  # text is of STRING data type
    response = cohere_client.embed(
        model='embed-english-light-v3.0',
        texts=[text], 
        input_type='search_document',
    )
    time.sleep(0.7) 
    return response.embeddings[0]
     


In [167]:
def chunk(text):
    # Initialize the text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Using 1000 characters as specified
        chunk_overlap=20,  # Optional overlap
        length_function=len,
        is_separator_regex=False,
    )

    # Create documents from the text
    texts = text_splitter.create_documents([text])

    # Extract the chunked text from each Document and store it in a list
    chunk_texts = [doc.page_content for doc in texts]

    return chunk_texts

In [168]:
def process_data(data_list):
    processed_data = []

    for data in data_list:
        # Chunk the transcript
        if data.get('transcript') is not None:
            chunks = chunk(data['transcript'])
            # Process each chunk
            for chunk_text in chunks:
                chunk_dict = {
                    "id": int(uuid.uuid4().int & ((1<<63)-1)),
                    "sourceURL": data['url'],
                    "videoURL": data['video'],
                    "chunkText": chunk_text,
                    "timestamp": int(time.time()),
                    "title": data['title'],
                    "date": int(datetime.strptime(data['date'], "%b %d, %Y").timestamp()),  # Correct format
                    "vector": get_embeddings(chunk_text)
                }
                processed_data.append(chunk_dict)

    return processed_data

In [169]:
data = [{
    "id": int(uuid.uuid4().int & (1<<63)-1),  # Ensure 64-bit integer
    "sourceURL": "example.com",
    "chunkText": "kubernetes cluster",
    "timestamp": int(time.time()),  # Current timestamp in Unix epoch
    "title": "Test",
    "date": int(datetime.strptime("19 Aug, 2024", "%d %b, %Y").timestamp()),  # Convert date to Unix epoch
    "videoURL": "https://www.youtube.com",  
    "vector": get_embeddings("kubernetes cluster")[0]
}]

In [170]:
processed_data = process_data(data_list)

In [177]:
print(processed_data[1]['chunkText'])

- Good afternoon everybody. I'd like to welcome you to
our next session titled, "The One About Control." Please help me welcome our speaker, Sri, Deputy CISO at Genpact, to the stage. (audience claps) - Thank you everyone. Thanks for making it here. I stand between you and Alicia Keys. I know. A tough job, right? She has a tough act to follow, isn't it? I take it that way. There's apparently some sort of similarity between two of us. We were born in the same month. Beyond that, I can't near
match her skills on stage. So you have to do with me for today. My name is Sriram. I go by Sri, make it easier
for everybody's life. What I'm gonna share today is my opinion. I would not like it to be
attributed to the employer, past, present, or future, but I'm hoping that
you'll enjoy the session. And what's with the title? Some people say titles don't matter. Sometimes they do, right? I'm assuming the presence


In [181]:
CLUSTER_ENDPOINT = zilliz_endpoint
TOKEN = zilliz_token

client = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN 
)

In [179]:
res = client.insert(
    collection_name="rsa_data",  # Replace with your actual collection name
    data=processed_data
)
print(res)

{'insert_count': 885, 'ids': [691852956980843724, 570248357860291200, 928189467412792789, 2679832169305268764, 3868603359910690928, 26878874009827340, 3422072761179498197, 120511222366938014, 3815471725113064092, 2882460686187935128, 1351885868410372438, 3285648081107961102, 2054168754872443786, 3569444816582847989, 3984962997506884840, 2527547204221160642, 89881643815025618, 1281061949847301966, 2209796347445042351, 3399336500577410990, 591170845484327217, 2942234106841452241, 1909904355331930867, 1539246471670856622, 3786082686428811058, 2750156607202413713, 3928263366308041256, 1683070817420162634, 4248916916338023843, 3852880119040006101, 2110353994926024799, 371884909595048256, 1905975541679787159, 3868220630336216039, 3066437210948358256, 2841588849441653180, 586080011585209945, 2963296628920750242, 4067643394066395927, 1027041791195534242, 1801298751737727822, 4477419276677459236, 122950766867784528, 663593837542856892, 3007318777024285575, 1309371449571586520, 58913123503558528

In [195]:
# Define the OAuth 2.0 scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate and Google Drive API setup
def authenticate_gdrive():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    print("Google Drive authenticated successfully.")
    return build('drive', 'v3', credentials=creds)

def download_image(service, file_id):
    print(f"Downloading image with file ID: {file_id}")
    request = service.files().get_media(fileId=file_id)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
    fh.seek(0)
    img = Image.open(fh)
    img.save(temp_file.name)
    print(f"Image downloaded and saved to: {temp_file.name}")
    return temp_file.name

def extract_folder_id_from_url(url):
    return url.split('/')[-1]

def get_image_descriptions(service, subfolder_url, json_filename="descriptions.json"):
    subfolder_id = extract_folder_id_from_url(subfolder_url)
    print(f"Processing images in Google Drive folder: {subfolder_url}")
    
    query = f"'{subfolder_id}' in parents and mimeType='image/png'"
    results = service.files().list(q=query, fields="files(id, name, webViewLink)").execute()
    items = results.get('files', [])
    
    # Load existing descriptions if the file exists
    if os.path.exists(json_filename):
        with open(json_filename, 'r') as f:
            descriptions = json.load(f)
    else:
        descriptions = {}
    
    for item in items:
        if item['webViewLink'] in descriptions:
            print(f"Skipping already processed image: {item['name']}")
            continue  # Skip already processed images

        print(f"Generating description for image: {item['name']}")
        local_image_path = download_image(service, item['id'])
        if local_image_path:
            try:
                with open(local_image_path, "rb") as image_file:
                    output = replicate.run(
                        "yorickvp/llava-13b:a0fdc44e4f2e1f20f2bb4e27846899953ac8e66c5886c5878fa1d6b73ce009e5",
                        input={
                            "image": image_file,
                            "prompt": """You are an expert in cybersecurity and technical communication. For each slide, generate a concise and 
                            informative description that focuses on the core content, key points, and implications of the information presented. 
                            Rather than merely identifying what is on the slide, explain the significance of the content, how it relates to cybersecurity 
                            practices, and any actionable insights or conclusions that can be drawn. Your response should be focused on 
                            conveying the essence of the slide's message and its relevance to the broader topic of cybersecurity. 
                            Provide the description in a single paragraph of no more than 200 words. If the slide is non-technical, such as an introduction, 
                            transition, or thank-you slide, simply reply with 'NA' and nothing else.""",
                            "max_tokens": 250
                        },
                        stream=False,
                    )
                    
                    response_text = ''.join([response for response in output])
                    descriptions[item['webViewLink']] = response_text
                    print(f"Description generated for image: {item['name']}")

                    # Append each description to the JSON file
                    with open(json_filename, 'w') as f:
                        json.dump(descriptions, f, indent=4)
                        print(f"Description saved to {json_filename}")
                    
            except Exception as e:
                print(f"Error processing image {item['name']}: {e}")
            finally:
                # Clean up the temporary file
                os.remove(local_image_path)
                print(f"Temporary file {local_image_path} deleted.")

    return descriptions

def process_images_for_vector_db(data_list):
    processed_data = []
    drive_service = authenticate_gdrive()

    for index, data in enumerate(data_list):
        print(f"\nProcessing item {index + 1}/{len(data_list)}: {data['title']}")
        if 'drive' in data:
            print(f"Found drive link: {data['drive']}")
            descriptions = get_image_descriptions(drive_service, data['drive'])
            
            for img_url, description in descriptions.items():
                print(f"Processing image: {img_url}")
                chunk_text = description[:1000]  # Truncate to 1000 characters
                embedding = get_embeddings(chunk_text)

                chunk_dict = {
                    "id": int(uuid.uuid4().int & ((1<<63)-1)),
                    "sourceURL": data['url'],
                    "imgURL": img_url,
                    "chunkText": chunk_text,
                    "timestamp": int(time.time()),
                    "title": data['title'],
                    "date": int(datetime.strptime(data['date'], "%b %d, %Y").timestamp()),
                    "vector": embedding
                }
                processed_data.append(chunk_dict)
                print(f"Image processed and data prepared for ingestion: {img_url}")

        else:
            print(f"No drive link found for item: {data['title']}")

    print("\nProcessing complete.")
    return processed_data


In [196]:
processed_images_data = process_images_for_vector_db(data_list)

Google Drive authenticated successfully.

Processing item 1/25: Closing Celebration with Alicia Keys
No drive link found for item: Closing Celebration with Alicia Keys

Processing item 2/25: The One About Controls
Found drive link: https://drive.google.com/drive/folders/1oh9WtzX8ZKkHjB_SjPxYuUFQR45U5i-1
Processing images in Google Drive folder: https://drive.google.com/drive/folders/1oh9WtzX8ZKkHjB_SjPxYuUFQR45U5i-1
Generating description for image: slide_12.png
Image downloaded and saved to: D:\temp\tmpqtblwc2g.png
Description generated for image: slide_12.png
Description saved to descriptions.json
Temporary file D:\temp\tmpqtblwc2g.png deleted.
Generating description for image: slide_11.png
Image downloaded and saved to: D:\temp\tmp0dmq76_k.png
Description generated for image: slide_11.png
Description saved to descriptions.json
Temporary file D:\temp\tmp0dmq76_k.png deleted.
Generating description for image: slide_10.png
Image downloaded and saved to: D:\temp\tmp9k64z42i.png
Descript

TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

In [200]:
# Define the OAuth 2.0 scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

def authenticate_gdrive():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    print("Google Drive authenticated successfully.")
    return build('drive', 'v3', credentials=creds)

def load_existing_descriptions(json_filename="descriptions.json"):
    if os.path.exists(json_filename):
        with open(json_filename, 'r') as f:
            return json.load(f)
    return {}

def download_image(service, file_id):
    print(f"Downloading image with file ID: {file_id}")
    request = service.files().get_media(fileId=file_id)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
    fh.seek(0)
    img = Image.open(fh)
    img.save(temp_file.name)
    print(f"Image downloaded and saved to: {temp_file.name}")
    return temp_file.name

def extract_folder_id_from_url(url):
    return url.split('/')[-1]

def get_image_descriptions(service, subfolder_url, existing_descriptions, json_filename="descriptions.json"):
    subfolder_id = extract_folder_id_from_url(subfolder_url)
    print(f"Processing images in Google Drive folder: {subfolder_url}")
    
    query = f"'{subfolder_id}' in parents and mimeType='image/png'"
    results = service.files().list(q=query, fields="files(id, name, webViewLink)").execute()
    items = results.get('files', [])
    
    descriptions = existing_descriptions.copy()  # Start with existing descriptions

    for item in items:
        if item['webViewLink'] in descriptions:
            print(f"Skipping already processed image: {item['name']}")
            continue  # Skip already processed images

        print(f"Generating description for image: {item['name']}")
        local_image_path = download_image(service, item['id'])
        if local_image_path:
            try:
                with open(local_image_path, "rb") as image_file:
                    output = replicate.run(
                        "yorickvp/llava-13b:a0fdc44e4f2e1f20f2bb4e27846899953ac8e66c5886c5878fa1d6b73ce009e5",
                        input={
                            "image": image_file,
                            "prompt": """You are an expert in cybersecurity and technical communication. For each slide, generate a concise and informative description that focuses on the core content, key points, and implications of the information presented. Rather than merely identifying what is on the slide, explain the significance of the content, how it relates to cybersecurity practices, and any actionable insights or conclusions that can be drawn. Your response should be focused on conveying the essence of the slide's message and its relevance to the broader topic of cybersecurity. Provide the description in a single paragraph of no more than 200 words. If the slide is non-technical, such as an introduction, transition, or thank-you slide, simply reply with 'NA' and nothing else.""",
                            "max_tokens": 250
                        },
                        stream=False,
                    )
                    
                    response_text = ''.join([response for response in output])
                    descriptions[item['webViewLink']] = response_text
                    print(f"Description generated for image: {item['name']}")

                    # Append each description to the JSON file
                    with open(json_filename, 'w') as f:
                        json.dump(descriptions, f, indent=4)
                        print(f"Description saved to {json_filename}")
                    
            except Exception as e:
                print(f"Error processing image {item['name']}: {e}")
            finally:
                # Clean up the temporary file
                os.remove(local_image_path)
                print(f"Temporary file {local_image_path} deleted.")

    return descriptions

def process_images_for_vector_db(data_list, start_index=5):
    processed_data = []
    drive_service = authenticate_gdrive()

    # Load existing descriptions
    existing_descriptions = load_existing_descriptions()

    for index, data in enumerate(data_list[start_index:], start=start_index + 1):
        print(f"\nProcessing item {index}/{len(data_list)}: {data['title']}")
        if 'drive' in data:
            print(f"Found drive link: {data['drive']}")
            descriptions = get_image_descriptions(drive_service, data['drive'], existing_descriptions)
            
            for img_url, description in descriptions.items():
                if img_url in existing_descriptions:
                    continue  # Skip already processed images

                print(f"Processing image: {img_url}")
                chunk_text = description[:1000]  # Truncate to 1000 characters
                embedding = get_embeddings(chunk_text)

                chunk_dict = {
                    "id": int(uuid.uuid4().int & ((1<<63)-1)),
                    "sourceURL": data['url'],
                    "imgURL": img_url,
                    "chunkText": chunk_text,
                    "timestamp": int(time.time()),
                    "title": data['title'],
                    "date": int(datetime.strptime(data['date'], "%b %d, %Y").timestamp()),
                    "vector": embedding
                }
                processed_data.append(chunk_dict)
                print(f"Image processed and data prepared for ingestion: {img_url}")

        else:
            print(f"No drive link found for item: {data['title']}")

    print("\nProcessing complete.")
    return processed_data




In [204]:
# Load existing processed data (if any)
existing_data = load_existing_descriptions()

# Begin processing from the 6th item (index 5)
processed_images_data = process_images_for_vector_db(data_list, start_index=6)

# Combine the existing data with new processed data
if existing_data:
    processed_images_data = existing_data + processed_images_data


Google Drive authenticated successfully.

Processing item 7/25: The ART of Probable: Test with AI, Atomic Red Team, and Threat Metrics
Found drive link: https://drive.google.com/drive/folders/1rM04BCx61K1xT4d8HAb_E-SV-6ud3Jg_
Processing images in Google Drive folder: https://drive.google.com/drive/folders/1rM04BCx61K1xT4d8HAb_E-SV-6ud3Jg_
Generating description for image: slide_16.png
Image downloaded and saved to: D:\temp\tmp59jg8jub.png
Description generated for image: slide_16.png
Description saved to descriptions.json
Temporary file D:\temp\tmp59jg8jub.png deleted.
Generating description for image: slide_15.png
Image downloaded and saved to: D:\temp\tmp5dn68ava.png
Description generated for image: slide_15.png
Description saved to descriptions.json
Temporary file D:\temp\tmp5dn68ava.png deleted.
Generating description for image: slide_14.png
Image downloaded and saved to: D:\temp\tmpwmnmgwij.png
Description generated for image: slide_14.png
Description saved to descriptions.json
Te

TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"